In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# Leer el archivo `model.txt` ignorando las primeras 3 líneas de metadatos
model = pd.read_csv(
    r'C:\\Users\\rass1\\OneDrive\\Escritorio\\calculo\\pandas\\model.txt',  # Cambiar a la ruta real del archivo
    sep='\\s+',  # Usar expresión regular para espacios
    skiprows=3  # Ignorar metadatos iniciales
)

# Corregir valores de HHMM para asegurar que tengan 4 dígitos
model['HHMM'] = model['HHMM'].astype(str).str.zfill(4)

# Crear columna de fecha combinando YYYYMMDD y HHMM
model['Timestamp'] = pd.to_datetime(
    model['YYYYMMDD'].astype(str) + model['HHMM'],
    format='%Y%m%d%H%M', errors='coerce'  # Usar errors='coerce' para manejar valores inválidos
)

# Verificar si hay valores NaT en la columna Timestamp
invalid_timestamps = model[model['Timestamp'].isna()]
if not invalid_timestamps.empty:
    print("Advertencia: Se encontraron valores de fecha/hora inválidos:")
    print(invalid_timestamps)
    # Eliminar filas con fechas/hora inválidas
    model.dropna(subset=['Timestamp'], inplace=True)

# Configurar el índice como Timestamp y eliminar columnas originales
model.set_index('Timestamp', inplace=True)
model.drop(columns=['YYYYMMDD', 'HHMM'], inplace=True)

# Verificar los datos cargados
print(model.head())

# Visualización inicial: Matriz de dispersión para las primeras 1000 filas
scatter_matrix(
    model.loc[model.index[:1000], 'M(m/s)':'D(deg)'],
    alpha=0.2, figsize=(10, 10), diagonal='kde'
)
plt.show()

# Histograma de velocidades del viento
model.loc[:, 'M(m/s)'].plot.hist(bins=np.arange(0, 35), edgecolor='black', alpha=0.7)
plt.title("Distribución de Velocidades del Viento (M(m/s))")
plt.xlabel("Velocidad (m/s)")
plt.ylabel("Frecuencia")
plt.show()

# Agregar columnas para 'year' y 'month'
model['month'] = model.index.month
model['year'] = model.index.year

# Agrupar por año y mes y calcular la media
monthly_data = model.groupby(by=['year', 'month']).mean()
monthly_data['ma'] = monthly_data['M(m/s)'].rolling(5, center=True).mean()

# Visualizar tendencias
monthly_data[['M(m/s)', 'ma']].plot(figsize=(15, 6))
plt.title("Tendencias Mensuales de Velocidades del Viento")
plt.xlabel("Tiempo")
plt.ylabel("Velocidad del Viento (m/s)")
plt.show()
